# Market Segmentation with Clustering - Lab

## Introduction

In this lab, you'll use your knowledge of clustering to perform market segmentation on a real-world dataset!

## Objectives

In this lab you will: 

- Use clustering to create and interpret market segmentation on real-world data 

## Getting Started

In this lab, you're going to work with the [Wholesale customers dataset](https://archive.ics.uci.edu/ml/datasets/wholesale+customers) from the UCI Machine Learning datasets repository. This dataset contains data on wholesale purchasing information from real businesses. These businesses range from small cafes and hotels to grocery stores and other retailers. 

Here's the data dictionary for this dataset:

|      Column      |                                               Description                                              |
|:----------------:|:------------------------------------------------------------------------------------------------------:|
|       FRESH      |                    Annual spending on fresh products, such as fruits and vegetables                    |
|       MILK       |                               Annual spending on milk and dairy products                               |
|      GROCERY     |                                   Annual spending on grocery products                                  |
|      FROZEN      |                                   Annual spending on frozen products                                   |
| DETERGENTS_PAPER |                  Annual spending on detergents, cleaning supplies, and paper products                  |
|   DELICATESSEN   |                           Annual spending on meats and delicatessen products                           |
|      CHANNEL     | Type of customer.  1=Hotel/Restaurant/Cafe, 2=Retailer. (This is what we'll use clustering to predict) |
|      REGION      |            Region of Portugal that the customer is located in. (This column will be dropped)           |



One benefit of working with this dataset for practice with segmentation is that we actually have the ground-truth labels of what market segment each customer actually belongs to. For this reason, we'll borrow some methodology from supervised learning and store these labels separately, so that we can use them afterward to check how well our clustering segmentation actually performed. 

Let's get started by importing everything we'll need.

In the cell below:

* Import `pandas`, `numpy`, and `matplotlib.pyplot`, and set the standard alias for each. 
* Use `numpy` to set a random seed of `0`.
* Set all matplotlib visualizations to appear inline.

In [5]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)  # For reproducibility
%matplotlib inline

Now, let's load our data and inspect it. You'll find the data stored in `'wholesale_customers_data.csv'`. 

In the cell below, load the data into a DataFrame and then display the first five rows to ensure everything loaded correctly.

In [6]:
raw_df = pd.read_csv('wholesale_customers_data.csv')
# Display the first few rows of the dataset 
raw_df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


Now, let's go ahead and store the `'Channel'` column in a separate variable and then drop both the `'Channel'` and `'Region'` columns. Then, display the first five rows of the new DataFrame to ensure everything worked correctly. 

In [7]:
channels = raw_df['Channel'].unique()
df = raw_df.copy()

Now, let's get right down to it and begin our clustering analysis. 

In the cell below:

* Import `KMeans` from `sklearn.cluster`, and then create an instance of it. Set the number of clusters to `2`
* Fit it to the data (`df`) 
* Get the predictions from the clustering algorithm and store them in `cluster_preds` 

In [27]:
#import KMeans from sklearn
from sklearn.cluster import KMeans


In [31]:
k_means = KMeans(n_clusters=2, random_state=42)
# Fit the model to the data
k_means.fit(df)
# Predict the clusters

cluster_preds = k_means.predict(df)

Now, use some of the metrics to check the performance. You'll use `calinski_harabasz_score()` and `adjusted_rand_score()`, which can both be found inside [`sklearn.metrics`](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation). 

In the cell below, import these scoring functions. 

In [32]:
from sklearn.metrics import calinski_harabasz_score, adjusted_rand_score

Now, start with CH score to get the variance ratio. 

In [39]:
#calculate the Calinski-Harabasz score
ch_score = calinski_harabasz_score(df, cluster_preds)
#Display the Calinski-Harabasz score
print(f'Calinski-Harabasz Score: {ch_score:.3f}')

Calinski-Harabasz Score: 171.685


Although you don't have any other numbers to compare this to, this is a pretty low score, suggesting that the clusters aren't great. 

Since you actually have ground-truth labels, in this case you can use `adjusted_rand_score()` to check how well the clustering performed. Adjusted Rand score is meant to compare two clusterings, which the score can interpret our labels as. This will tell us how similar the predicted clusters are to the actual channels. 

Adjusted Rand score is bounded between -1 and 1. A score close to 1 shows that the clusters are almost identical. A score close to 0 means that predictions are essentially random, while a score close to -1 means that the predictions are pathologically bad, since they are worse than random chance. 

In the cell below, call `adjusted_rand_score()` and pass in `channels` and `cluster_preds` to see how well your first iteration of clustering performed. 

In [42]:
#Calculate the Adjusted Rand Index
ari_score = adjusted_rand_score(raw_df['Channel'], cluster_preds)
#Display the Adjusted Rand Index
print(f'Adjusted Rand Index: {ari_score:.3f}')

Adjusted Rand Index: -0.031


According to these results, the clusterings were essentially no better than random chance. Let's see if you can improve this. 

### Scaling our dataset

Recall that k-means clustering is heavily affected by scaling. Since the clustering algorithm is distance-based, this makes sense. Let's use `StandardScaler` to scale our dataset and then try our clustering again and see if the results are different. 

In the cells below:

* Import and instantiate [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and use it to transform the dataset  
* Instantiate and fit k-means to this scaled data, and then use it to predict clusters 
* Calculate the adjusted Rand score for these new predictions 

In [45]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

In [47]:
scaled_k_means = KMeans(n_clusters=2, random_state=42)

scaled_preds = scaled_k_means.fit_predict(scaled_df)

In [54]:
#Evaluate the clustering performance on the scaled data
scaled_ch_score = calinski_harabasz_score(scaled_df, scaled_preds)
scaled_ari_score = adjusted_rand_score(raw_df['Channel'], scaled_preds)
#Display the scores for the scaled data
print(f'Scaled Calinski-Harabasz Score: {scaled_ch_score:.3f}')
print(f'Scaled Adjusted Rand Index: {scaled_ari_score:.3f}')

Scaled Calinski-Harabasz Score: 154.899
Scaled Adjusted Rand Index: 0.927


That's a big improvement! Although it's not perfect, we can see that scaling our data had a significant effect on the quality of our clusters. 

## Incorporating PCA

Since clustering algorithms are distance-based, this means that dimensionality has a definite effect on their performance. The greater the dimensionality of the dataset, the greater the total area that we have to worry about our clusters existing in. Let's try using Principal Component Analysis to transform our data and see if this affects the performance of our clustering algorithm. 

Since you've already seen PCA in a previous section, we will let you figure this out by yourself. 

In the cells below:

* Import [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) from the appropriate module in sklearn 
* Create a `PCA` instance and use it to transform our scaled data  
* Investigate the explained variance ratio for each Principal Component. Consider dropping certain components to reduce dimensionality if you feel it is worth the loss of information 
* Create a new `KMeans` object, fit it to our PCA-transformed data, and check the adjusted Rand score of the predictions it makes. 

**_NOTE:_** Your overall goal here is to get the highest possible adjusted Rand score. Don't be afraid to change parameters and rerun things to see how it changes. 

In [55]:
from sklearn.decomposition import PCA

In [60]:
#instantiate PCA
pca = PCA(n_components=2)
# Fit and transform the scaled data
pca_df = pca.fit_transform(scaled_df)
# Create a DataFrame from the PCA results
pca_df = pd.DataFrame(data=pca_df, columns=['PC1', 'PC2'])
# Add the cluster predictions to the PCA DataFrame
pca_df['Cluster'] = scaled_preds

In [62]:
#check the explained variance ratio
print(f'Explained Variance Ratio: {pca.explained_variance_ratio_}')
print(f'Total Explained Variance: {np.sum(pca.explained_variance_ratio_):.3f}')

Explained Variance Ratio: [0.38750123 0.22374588]
Total Explained Variance: 0.611


In [64]:
#fit the KMeans model on the PCA data
pca_k_means = KMeans(n_clusters=2, random_state=42)
# Predict the clusters using PCA data
pca_preds = pca_k_means.fit_predict(pca_df[['PC1', 'PC2']])

In [67]:
#evaluate the clustering performance on the PCA data
pca_ch_score = calinski_harabasz_score(pca_df[['PC1', 'PC2']], pca_preds)
pca_ari_score = adjusted_rand_score(raw_df['Channel'], pca_preds)
#Display the scores for the PCA data
print(f'PCA Calinski-Harabasz Score: {pca_ch_score:.3f}')
print(f'PCA Adjusted Rand Index: {pca_ari_score:.3f}')

PCA Calinski-Harabasz Score: 298.730
PCA Adjusted Rand Index: 0.822


**_Question_**:  What was the Highest Adjusted Rand Score you achieved? Interpret this score and determine the overall quality of the clustering. Did PCA affect the performance overall?  How many principal components resulted in the best overall clustering performance? Why do you think this is?

Write your answer below this line:

__
*The best clustering performance was achieved after scaling the data, with an Adjusted Rand Score of 0.927, indicating excellent alignment with the true labels.*

*Raw data: ARI = -0.031 (worse than random)*

*Scaled data: ARI = 0.927 (strong clustering)*

*PCA-transformed data: ARI = 0.822 (good, but slightly lower)*

*Scaling had the greatest positive impact. PCA helped but slightly reduced performance, likely due to loss of some informative features.*
_____________________________________________________________________________________________________________________________

## Optional (Level up) 

### Hierarchical Agglomerative Clustering

Now that we've tried doing market segmentation with k-means clustering, let's end this lab by trying with HAC!

In the cells below, use [Agglomerative clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) to make cluster predictions on the datasets we've created and see how HAC's performance compares to k-mean's performance. 

**_NOTE_**: Don't just try HAC on the PCA-transformed dataset -- also compare algorithm performance on the scaled and unscaled datasets, as well! 

In [73]:
from sklearn.cluster import AgglomerativeClustering

In [78]:
#HAC on raw data
hac_raw = AgglomerativeClustering(n_clusters=2)
# Fit the model to the raw data
hac_raw_preds = hac_raw.fit_predict(df)
# Evaluate the clustering performance on the raw data
hac_raw_ch_score = calinski_harabasz_score(df, hac_raw_preds)
hac_raw_ari_score = adjusted_rand_score(raw_df['Channel'], hac_raw_preds)
# Display the scores for the raw data
print(f'HAC Raw Calinski-Harabasz Score: {hac_raw_ch_score:.3f}')
print(f'HAC Raw Adjusted Rand Index: {hac_raw_ari_score:.3f}')

HAC Raw Calinski-Harabasz Score: 147.456
HAC Raw Adjusted Rand Index: -0.019


In [87]:
#HAC on scaled data
hac_scaled = AgglomerativeClustering(n_clusters=2)
# Fit the model to the scaled data
hac_scaled_preds = hac_scaled.fit_predict(scaled_df)
# Evaluate the clustering performance on the scaled data
hac_scaled_ch_score = calinski_harabasz_score(scaled_df, hac_scaled_preds)
hac_scaled_ari_score = adjusted_rand_score(raw_df['Channel'], hac_scaled_preds)
# Display the scores for the scaled data
print(f'HAC Scaled Calinski-Harabasz Score: {hac_scaled_ch_score:.3f}')
print(f'HAC Scaled Adjusted Rand Index: {hac_scaled_ari_score:.3f}')

HAC Scaled Calinski-Harabasz Score: 149.648
HAC Scaled Adjusted Rand Index: 0.982


In [94]:
#HAC on PCA data
hac_pca = AgglomerativeClustering(n_clusters=2)
# Fit the model to the PCA data
hac_pca_preds = hac_pca.fit_predict(pca_df[['PC1', 'PC2']])
# Evaluate the clustering performance on the PCA data
hac_pca_ch_score = calinski_harabasz_score(pca_df[['PC1', 'PC2']], hac_pca_preds)
hac_pca_ari_score = adjusted_rand_score(raw_df['Channel'], hac_pca_preds)
# Display the scores for the PCA data
print(f'HAC PCA Calinski-Harabasz Score: {hac_pca_ch_score:.3f}')
print(f'HAC PCA Adjusted Rand Index: {hac_pca_ari_score:.3f}')

HAC PCA Calinski-Harabasz Score: 280.822
HAC PCA Adjusted Rand Index: 0.858


*Raw Data: Performed poorly (negative ARI), confirming that scaling is essential for distance-based clustering algorithms like HAC.*

*Scaled Data: Delivered the highest ARI (0.982), showing near-perfect alignment with true labels. Scaling clearly optimized HAC performance.*

*PCA Data: Had the highest CH score, suggesting well-separated clusters, but the ARI (0.858) slightly dropped — indicating that some label-relevant information was lost in dimensionality reduction.*

## Summary

In this lab, you used your knowledge of clustering to perform a market segmentation on a real-world dataset. You started with a cluster analysis with poor performance, and then implemented some changes to iteratively improve the performance of the clustering analysis!